#### **Hospital Occupancy Trend**

In [2]:
from pyspark.sql import functions as F

df_adm = spark.table("silver_admissions")

gold_occupancy = (
    df_adm
    .groupBy("State", "WeekDate")
    .agg(
        F.round(F.avg("AvgOccupancyRate"), 2).alias("AvgOccupancyRate"),
        F.round(F.avg("AvgICUOccupancyRate"), 2).alias("AvgICUOccupancyRate"),
        F.round(F.avg("AvgBedsUsed"), 0).alias("AvgBedsUsed"),
        F.round(F.avg("AvgICUBedsUsed"), 0).alias("AvgICUBedsUsed"),
        F.sum("TotalCovidAdmissions").alias("TotalCovidAdmissions"),
        F.sum("TotalInfluenzaAdmissions").alias("TotalInfluenzaAdmissions")
    )
    .orderBy("WeekDate")
)

gold_occupancy.write.format("delta").mode("overwrite").saveAsTable("gold_occupancy_trend")
print("✅ gold_occupancy_trend saved")

StatementMeta(, 2750d485-490d-4798-ba7c-d04dda1091c7, 4, Finished, Available, Finished, False)

✅ gold_occupancy_trend saved


#### **Staff Utilisation by Department**

In [3]:
df_staff = spark.table("silver_staff")

gold_staff = (
    df_staff
    .groupBy("Department", "ShiftDate")
    .agg(
        F.countDistinct("StaffID").alias("StaffOnDuty"),
        F.sum("HoursWorked").alias("TotalHoursWorked"),
        F.count(F.when(F.col("Role") == "Doctor", True)).alias("DoctorsOnDuty"),
        F.count(F.when(F.col("Role") == "Nurse", True)).alias("NursesOnDuty")
    )
    .orderBy("ShiftDate", "Department")
)

gold_staff.write.format("delta").mode("overwrite").saveAsTable("gold_staff_utilisation")
print("✅ gold_staff_utilisation saved")

StatementMeta(, 2750d485-490d-4798-ba7c-d04dda1091c7, 5, Finished, Available, Finished, False)

✅ gold_staff_utilisation saved


#### **Drug Inventory Risk**

In [4]:
df_drugs = spark.table("silver_drugs")

gold_drugs = (
    df_drugs
    .withColumn("RiskFlag",
        F.when(F.col("StockPct") < 20, "🔴 Critical")
         .when(F.col("StockPct") < 40, "🟠 Low")
         .when(F.col("StockPct") < 60, "🟡 Medium")
         .otherwise("🟢 Adequate"))
    .withColumn("DaysToStockout",
        F.when(F.col("StockLevel") < F.col("ReorderThreshold"),
            F.lit("Reorder Now"))
         .otherwise(F.lit("OK")))
    .withColumn("StockValue",
        F.round(F.col("StockLevel") * F.col("UnitCost"), 2))
    .select("DrugID", "DrugName", "Category", "Department",
            "StockLevel", "MaxCapacity", "StockPct",
            "RiskFlag", "DaysToStockout", "StockValue", "LastRestocked")
)

gold_drugs.write.format("delta").mode("overwrite").saveAsTable("gold_drug_inventory")
print("✅ gold_drug_inventory saved")

StatementMeta(, 2750d485-490d-4798-ba7c-d04dda1091c7, 6, Finished, Available, Finished, False)

✅ gold_drug_inventory saved


#### **Department Summary (joined gold table)**

In [5]:
df_staff_summary = spark.table("gold_staff_utilisation")
df_drug_summary = spark.table("gold_drug_inventory")

# Staff summary per department
dept_staff = (
    df_staff_summary
    .groupBy("Department")
    .agg(
        F.round(F.avg("StaffOnDuty"), 1).alias("AvgDailyStaff"),
        F.round(F.avg("TotalHoursWorked"), 1).alias("AvgDailyHours")
    )
)

# Drug risk per department
dept_drugs = (
    df_drug_summary
    .groupBy("Department")
    .agg(
        F.count("DrugID").alias("TotalDrugs"),
        F.sum(F.when(F.col("RiskFlag").contains("Critical"), 1).otherwise(0)).alias("CriticalDrugs"),
        F.round(F.sum("StockValue"), 2).alias("TotalStockValue")
    )
)

# Join both
gold_dept_summary = dept_staff.join(dept_drugs, on="Department", how="outer")

gold_dept_summary.write.format("delta").mode("overwrite").saveAsTable("gold_department_summary")
print("✅ gold_department_summary saved")

StatementMeta(, 2750d485-490d-4798-ba7c-d04dda1091c7, 7, Finished, Available, Finished, False)

✅ gold_department_summary saved


In [6]:
display(spark.sql("SHOW TABLES"))

StatementMeta(, 2750d485-490d-4798-ba7c-d04dda1091c7, 8, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 55dd411c-bfd2-4cf7-9eec-bf4a4423cadb)